In [1]:
import numpy as np
import os
import tensorflow as tf
import sklearn.metrics as sk
from sklearn.model_selection import KFold

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ###### 

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Model functions

In [2]:
#transform to one_hot array
def one_hot(input_data):
    one_hot = []
    for item in input_data:
        if item == 0.:
            one_h = [1.,0.,0.,0.,0.]
        elif item == 1.:
            one_h = [0.,1.,0.,0.,0.]
        elif item == 2.:
            one_h = [0.,0.,1.,0.,0.]
        elif item == 3.:
            one_h = [0.,0.,0.,1.,0.]
        elif item == 4.:
            one_h = [0.,0.,0.,0.,1.]

        one_hot.append(one_h)
    one_hot = np.array(one_hot)
    #one_hot = one_hot.reshape(len(one_hot),10,1)
    #one_hot = one_hot.reshape(len(one_hot), 7,1)
    #return tf.constant([one_hot])
    return one_hot

def modelInput(input, INPUT_NODE, NEURONS, reuse, keep_prob):
    # Create variable named "weights".
    weights = get_scope_variable("DNN", "weights", shape=[INPUT_NODE, NEURONS], initializer= tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_AVG', uniform=True), reuse = reuse)
    # Create variable named "biases".
    biases = get_scope_variable("DNN", "biases", shape=[NEURONS], initializer= tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_AVG', uniform=True), reuse = reuse)
    hidden1 = tf.nn.elu(tf.matmul(input, weights) + biases)
    dropout1 = tf.nn.dropout(hidden1,keep_prob)
    return dropout1

def modelhHidden(input, INPUT_NODE, NEURONS, reuse, keep_prob):
    # Create variable named "weights".
    weights = get_scope_variable("DNN", "weights", shape=[NEURONS, NEURONS], initializer= tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_AVG', uniform=True), reuse = reuse)
    # Create variable named "biases".
    biases = get_scope_variable("DNN", "biases", shape=[NEURONS], initializer= tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_AVG', uniform=True), reuse = reuse)
    hidden = tf.nn.elu(tf.matmul(input, weights) + biases)
    dropout = tf.nn.dropout(hidden,keep_prob)
    return dropout

def modelOutput(input, INPUT_NODE, NEURONS, reuse):
    # Create variable named "weights".
    weights = get_scope_variable("DNN", "weights", shape=[NEURONS, 5], initializer= tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_AVG', uniform=True), reuse = reuse)
    # Create variable named "biases".
    biases = get_scope_variable("DNN", "biases", shape=[5], initializer= tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_AVG', uniform=True), reuse = reuse)
    output = tf.matmul(input, weights) + biases
    output_after_softmax = tf.nn.softmax(output)
    return output, output_after_softmax

def training(batch_xs,INPUT_NODE,NEURONS, reuse, keep_prob):
    sess = tf.InteractiveSession()
    with tf.variable_scope("layer1",reuse = reuse):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        output1 = modelInput(batch_xs, INPUT_NODE, NEURONS, reuse, keep_prob)
    with tf.variable_scope("layer2",reuse = reuse):
        output2 = modelhHidden(output1, INPUT_NODE, NEURONS, reuse, keep_prob)
    with tf.variable_scope("layer3",reuse = reuse):
        output3 = modelhHidden(output2, INPUT_NODE, NEURONS, reuse, keep_prob)
    with tf.variable_scope("layer4",reuse = reuse):
        output4 = modelhHidden(output3, INPUT_NODE, NEURONS, reuse, keep_prob)
    with tf.variable_scope("layer5",reuse = reuse):
        output5 = modelhHidden(output4, INPUT_NODE, NEURONS, reuse, keep_prob)
    with tf.variable_scope("output",reuse = reuse):
        return modelOutput(output5, INPUT_NODE, NEURONS, reuse)
    
def get_scope_variable(scope_name, var, shape, initializer, reuse):
    with tf.variable_scope(scope_name,reuse = reuse) as scope:
        try:
            v = tf.get_variable(var, shape=shape, initializer=initializer)
        except ValueError:
            scope.reuse_variables()
            v = tf.get_variable(var, shape=shape, initializer=initializer)
    return v

Training and cross validation step

In [3]:
sess = tf.InteractiveSession()
#preprocess the data

#combine train and validation set
X_Cross = tf.concat([X_train1,X_valid1],0) #shape = (30596,784)
y_Cross = tf.concat([y_train1,y_valid1],0)
kf = KFold(n_splits=10,shuffle=True)
kf.get_n_splits(X_Cross)
flod = 0

Cross_valid_flod_accuracy = []
dropout_accuracy = []
Test_accuracy = []

for dropout_rate in range(5, 11, 1):
    print('--------------------')
    print('Dropout keep rate: %f' %(dropout_rate/10))
    print('Training with 10-flod cross validation')
    print('--------------------')
    flod = 0
    for train_index, valid_index in kf.split(sess.run(X_Cross)):
        print('Flod number: %d'%(flod+1))
        flod += 1
        X_train, X_valid = sess.run(X_Cross)[train_index], sess.run(X_Cross)[valid_index]
        y_train, y_valid = sess.run(y_Cross)[train_index], sess.run(y_Cross)[valid_index]

        y_train =  one_hot(y_train)
        y_valid =  one_hot(y_valid)

        #define model
        INPUT_NODE = 784
        NEURONS = 128
        OUTPUT_NODE = 5

        #define loss (cost) function
        true_labels = tf.placeholder(tf.int32, [None,5])
        x = tf.placeholder(tf.float32, [None,INPUT_NODE])
        dropout_keep_prob = tf.placeholder(tf.float32)

        if(flod-1 == 0):
            reuse = False
        else:
            reuse = True

        output, output_after_softmax = training(x,INPUT_NODE,NEURONS,reuse,dropout_keep_prob)

        #better take unsoftmax output as logits as our cross entropy loss function will do softmax
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits= output,labels= tf.argmax(true_labels, 1))) 

        with tf.variable_scope('Adam',reuse=tf.AUTO_REUSE) as scope:
            train_step = tf.train.AdamOptimizer(0.01).minimize(loss)


        correct_prediction = tf.equal(tf.argmax(output_after_softmax, 1), tf.argmax(true_labels, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        #Training and Testing

        # define training parameters
        training_epochs = 15
        batch_size = 500
        train_data_size = np.prod(X_train.shape[0]) #28038
        total_batch = int(train_data_size/batch_size)

        # prepare session
        sess = tf.InteractiveSession()
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()

        # Training cycle
        current_best = 0.0
        stopping_step = 0
        TP_0 = 0
        FP_0 = 0
        FN_0 = 0
        TP_1 = 0
        FP_1 = 0
        FN_1 = 0
        TP_2 = 0
        FP_2 = 0
        FN_2 = 0
        TP_3 = 0
        FP_3 = 0
        FN_3 = 0
        TP_4 = 0
        FP_4 = 0
        FN_4 = 0
        precision_0 = 0.0
        recall_0 = 0.0
        precision_1 = 0.0
        recall_1 = 0.0
        precision_2 = 0.0
        recall_2 = 0.0
        precision_3 = 0.0
        recall_3 = 0.0
        precision_4 = 0.0
        recall_4 = 0.0
        current_best = 0.0
        epoch_number = 0;
        for epoch in range(training_epochs):
            #print ('epoch %f in %f'% (epoch+1, training_epochs))
            epoch_number+=1
            # generate random indexes
            indexes = np.random.permutation(train_data_size)

            # run all datas for one epoch
            for position in range(0, train_data_size, batch_size):
                # generate mini batch ids
                ids = indexes[position:(position+batch_size) if (position+batch_size) < train_data_size else train_data_size]
                batch_xs = X_train[ids]
                batch_ts = y_train[ids]

                sess.run(train_step,feed_dict={x:batch_xs, true_labels:batch_ts, dropout_keep_prob: dropout_rate/10})

            # now we test model per epoch
            loss_val, acc_val = sess.run([loss, accuracy],feed_dict={x:X_valid, true_labels:y_valid, dropout_keep_prob: 1})
            predicted_result = sess.run(output_after_softmax,feed_dict={x:X_valid, true_labels:y_valid, dropout_keep_prob: 1})

            fianl_predict = np.argmax(predicted_result,1)
            labels = np.argmax(y_valid,1)
            
            TP_0 = 0
            FP_0 = 0
            FN_0 = 0
            TP_1 = 0
            FP_1 = 0
            FN_1 = 0
            TP_2 = 0
            FP_2 = 0
            FN_2 = 0
            TP_3 = 0
            FP_3 = 0
            FN_3 = 0
            TP_4 = 0
            FP_4 = 0
            FN_4 = 0
            
            
            for i in range(0,fianl_predict.size,1):
                if(fianl_predict[i] == 0):
                    if(labels[i] == 0):
                        TP_0+=1
                    else:
                        FP_0+=1
                if(fianl_predict[i] == 1):
                    if(labels[i] == 1):
                        TP_1+=1
                    else:
                        FP_1+=1
                if(fianl_predict[i] == 2):
                    if(labels[i] == 2):
                        TP_2+=1
                    else:
                        FP_2+=1
                if(fianl_predict[i] == 3):
                    if(labels[i] == 3):
                        TP_3+=1
                    else:
                        FP_3+=1
                if(fianl_predict[i] == 4):
                    if(labels[i] == 4):
                        TP_4+=1
                    else:
                        FP_4+=1
                if(labels[i] == 0):
                    if(fianl_predict[i] != 0):
                        FN_0+=1
                if (labels[i] == 1):
                    if(fianl_predict[i] != 1):
                        FN_1+=1
                if (labels[i] == 2):
                    if(fianl_predict[i] != 2):
                        FN_2+=1
                if (labels[i] == 3):
                    if(fianl_predict[i] != 3):
                        FN_3+=1
                if (labels[i] == 4):
                    if(fianl_predict[i] != 4):
                        FN_4+=1

            #print ('epoch: %d, Loss: %f, Accuracy: %f'% (epoch+1, loss_val, acc_val))

            if(acc_val > current_best):
                current_best = acc_val
                stopping_step = 0
            else:
                stopping_step +=1
                
            if(stopping_step >= 3):
                break
                
        precision_0 = TP_0/(TP_0 + FP_0)
        recall_0 = TP_0/(TP_0 + FN_0)
        precision_1 = TP_1/(TP_1 + FP_1)
        recall_1 = TP_1/(TP_1 + FN_1)
        precision_2 = TP_2/(TP_2 + FP_2)
        recall_2 = TP_2/(TP_2 + FN_2)
        precision_3 = TP_3/(TP_3 + FP_3)
        recall_3 = TP_3/(TP_3 + FN_3)
        precision_4 = TP_4/(TP_4 + FP_4)
        recall_4 = TP_4/(TP_4 + FN_4)
        print ('Stop at epoch %d, Final accuracy: %f'% (epoch_number-1,current_best))
        print ('Label 0 Precision: %f, Recall: %f'%(precision_0,recall_0))
        print ('Label 1 Precision: %f, Recall: %f'%(precision_1,recall_1))
        print ('Label 2 Precision: %f, Recall: %f'%(precision_2,recall_2))
        print ('Label 3 Precision: %f, Recall: %f'%(precision_3,recall_3))
        print ('Label 4 Precision: %f, Recall: %f'%(precision_4,recall_4))
        Cross_valid_flod_accuracy.append(current_best)
        
                
    Cross_valid_avg_accuracy = np.array(Cross_valid_flod_accuracy)
    print('Average accuracy of 10 flod cross validation: %f' %np.mean(Cross_valid_avg_accuracy))
    dropout_accuracy.append(np.mean(Cross_valid_avg_accuracy))

dropout_accuracy_best = np.array(dropout_accuracy)
print('-------------------------------------------------------------------')
print('Best dropout keep rate after 10-flod cross validation: %f' %((np.argmax(dropout_accuracy_best)+5)/10))
print('-------------------------------------------------------------------')
saver.save(sess, './model/dnn_session/Team46_HW2.ckpt')
sess.close()

--------------------
Dropout keep rate: 0.500000
Training with 10-flod cross validation
--------------------
Flod number: 1
Stop at epoch 6, Final accuracy: 0.972876
Label 0 Precision: 1.000000, Recall: 0.975845
Label 1 Precision: 0.989474, Recall: 0.973373
Label 2 Precision: 0.904610, Recall: 0.979346
Label 3 Precision: 0.989691, Recall: 0.953642
Label 4 Precision: 0.972318, Recall: 0.972318
Flod number: 2
Stop at epoch 8, Final accuracy: 0.979085
Label 0 Precision: 0.965404, Recall: 0.998296
Label 1 Precision: 0.995595, Recall: 0.975540
Label 2 Precision: 0.994253, Recall: 0.902609
Label 3 Precision: 0.921450, Recall: 0.995106
Label 4 Precision: 0.984694, Recall: 0.981356
Flod number: 3
Stop at epoch 7, Final accuracy: 0.982026
Label 0 Precision: 0.998192, Recall: 0.980462
Label 1 Precision: 0.998514, Recall: 0.958631
Label 2 Precision: 0.906203, Recall: 0.993367
Label 3 Precision: 0.991304, Recall: 0.956376
Label 4 Precision: 0.989967, Recall: 0.991625
Flod number: 4
Stop at epoch 8

Stop at epoch 14, Final accuracy: 0.989212
Label 0 Precision: 0.985437, Recall: 0.995098
Label 1 Precision: 0.992847, Recall: 0.995696
Label 2 Precision: 0.978333, Recall: 0.984899
Label 3 Precision: 0.995058, Recall: 0.974194
Label 4 Precision: 0.994393, Recall: 0.996255
Flod number: 9
Stop at epoch 11, Final accuracy: 0.989539
Label 0 Precision: 0.991424, Recall: 0.996552
Label 1 Precision: 0.994286, Recall: 0.992867
Label 2 Precision: 0.984746, Recall: 0.974832
Label 3 Precision: 0.984899, Recall: 0.979967
Label 4 Precision: 0.984746, Recall: 0.996569
Flod number: 10
Stop at epoch 8, Final accuracy: 0.983655
Label 0 Precision: 0.981132, Recall: 0.987910
Label 1 Precision: 0.988321, Recall: 0.988321
Label 2 Precision: 0.979167, Recall: 0.957555
Label 3 Precision: 0.963141, Recall: 0.985246
Label 4 Precision: 0.986464, Recall: 0.978188
Average accuracy of 10 flod cross validation: 0.983582
--------------------
Dropout keep rate: 0.800000
Training with 10-flod cross validation
--------

Stop at epoch 11, Final accuracy: 0.988235
Label 0 Precision: 0.983793, Recall: 0.990212
Label 1 Precision: 0.995763, Recall: 0.984637
Label 2 Precision: 0.964103, Recall: 0.992958
Label 3 Precision: 0.989899, Recall: 0.986577
Label 4 Precision: 0.989209, Recall: 0.970018
Flod number: 6
Stop at epoch 6, Final accuracy: 0.982680
Label 0 Precision: 0.991667, Recall: 0.972222
Label 1 Precision: 0.981402, Recall: 0.995646
Label 2 Precision: 0.978512, Recall: 0.972085
Label 3 Precision: 0.993266, Recall: 0.976821
Label 4 Precision: 0.964413, Recall: 0.992674
Flod number: 7
Stop at epoch 13, Final accuracy: 0.990520
Label 0 Precision: 0.996587, Recall: 0.986486
Label 1 Precision: 0.981453, Recall: 0.998428
Label 2 Precision: 0.983498, Recall: 0.986755
Label 3 Precision: 0.996732, Recall: 0.980707
Label 4 Precision: 0.986842, Recall: 0.991736
Flod number: 8
Stop at epoch 14, Final accuracy: 0.991827
Label 0 Precision: 0.996479, Recall: 0.984348
Label 1 Precision: 0.998221, Recall: 0.855183
La

Testing step

In [4]:
#define model
INPUT_NODE = 784
NEURONS = 128
OUTPUT_NODE = 5

#define loss (cost) function
true_labels = tf.placeholder(tf.int32, [None,5])
x = tf.placeholder(tf.float32, [None,INPUT_NODE])
dropout_keep_prob = tf.placeholder(tf.float32)

output, output_after_softmax = training(x,INPUT_NODE,NEURONS,False,1.0)

#better take unsoftmax output as logits as our cross entropy loss function will do softmax
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits= output,labels= tf.argmax(true_labels, 1))) 

with tf.variable_scope('Adam',reuse=tf.AUTO_REUSE) as scope:
    train_step = tf.train.AdamOptimizer(0.01).minimize(loss)


correct_prediction = tf.equal(tf.argmax(output_after_softmax, 1), tf.argmax(true_labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# define training parameters
training_epochs = 15
batch_size = 500
train_data_size = np.prod(X_train1.shape[0]) #28038
total_batch = int(train_data_size/batch_size)

sess2 = tf.InteractiveSession()
sess2.run(tf.global_variables_initializer())
saver2 = tf.train.Saver()

# Training cycle
current_best = 0.0
stopping_step = 0
TP_0 = 0
FP_0 = 0
FN_0 = 0
TP_1 = 0
FP_1 = 0
FN_1 = 0
TP_2 = 0
FP_2 = 0
FN_2 = 0
TP_3 = 0
FP_3 = 0
FN_3 = 0
TP_4 = 0
FP_4 = 0
FN_4 = 0
precision_0 = 0.0
recall_0 = 0.0
precision_1 = 0.0
recall_1 = 0.0
precision_2 = 0.0
recall_2 = 0.0
precision_3 = 0.0
recall_3 = 0.0
precision_4 = 0.0
recall_4 = 0.0
epoch_number = 0;
y_train1 =  one_hot(y_train1)
y_test1 =  one_hot(y_test1)
for epoch in range(training_epochs):
    #print ('epoch %f in %f'% (epoch+1, training_epochs))
    epoch_number+=1
    # generate random indexes
    indexes = np.random.permutation(train_data_size)


    # run all datas for one epoch
    for position in range(0, train_data_size, batch_size):
        # generate mini batch ids
        ids = indexes[position:(position+batch_size) if (position+batch_size) < train_data_size else train_data_size]
        batch_xs = X_train1[ids]
        batch_ts = y_train1[ids]

        sess2.run(train_step,feed_dict={x:batch_xs, true_labels:batch_ts, dropout_keep_prob: 1})

    loss_val, acc_val = sess2.run([loss, accuracy],feed_dict={x:X_test1, true_labels:y_test1, dropout_keep_prob: 1})
    predicted_result = sess2.run(output_after_softmax,feed_dict={x:X_test1, true_labels:y_test1, dropout_keep_prob: 1})

    fianl_predict = np.argmax(predicted_result,1)
    labels = np.argmax(y_test1,1)
    
    if(acc_val > current_best):
        current_best = acc_val

    TP_0 = 0
    FP_0 = 0
    FN_0 = 0
    TP_1 = 0
    FP_1 = 0
    FN_1 = 0
    TP_2 = 0
    FP_2 = 0
    FN_2 = 0
    TP_3 = 0
    FP_3 = 0
    FN_3 = 0
    TP_4 = 0
    FP_4 = 0
    FN_4 = 0


    for i in range(0,fianl_predict.size,1):
        if(fianl_predict[i] == 0):
            if(labels[i] == 0):
                TP_0+=1
            else:
                FP_0+=1
        if(fianl_predict[i] == 1):
            if(labels[i] == 1):
                TP_1+=1
            else:
                FP_1+=1
        if(fianl_predict[i] == 2):
            if(labels[i] == 2):
                TP_2+=1
            else:
                FP_2+=1
        if(fianl_predict[i] == 3):
            if(labels[i] == 3):
                TP_3+=1
            else:
                FP_3+=1
        if(fianl_predict[i] == 4):
            if(labels[i] == 4):
                TP_4+=1
            else:
                FP_4+=1
        if(labels[i] == 0):
            if(fianl_predict[i] != 0):
                FN_0+=1
        if (labels[i] == 1):
            if(fianl_predict[i] != 1):
                FN_1+=1
        if (labels[i] == 2):
            if(fianl_predict[i] != 2):
                FN_2+=1
        if (labels[i] == 3):
            if(fianl_predict[i] != 3):
                FN_3+=1
        if (labels[i] == 4):
            if(fianl_predict[i] != 4):
                FN_4+=1

precision_0 = TP_0/(TP_0 + FP_0)
recall_0 = TP_0/(TP_0 + FN_0)
precision_1 = TP_1/(TP_1 + FP_1)
recall_1 = TP_1/(TP_1 + FN_1)
precision_2 = TP_2/(TP_2 + FP_2)
recall_2 = TP_2/(TP_2 + FN_2)
precision_3 = TP_3/(TP_3 + FP_3)
recall_3 = TP_3/(TP_3 + FN_3)
precision_4 = TP_4/(TP_4 + FP_4)
recall_4 = TP_4/(TP_4 + FN_4)
print ('Test accuracy: %f'% (current_best))
print ('Label 0 Precision: %f, Recall: %f'%(precision_0,recall_0))
print ('Label 1 Precision: %f, Recall: %f'%(precision_1,recall_1))
print ('Label 2 Precision: %f, Recall: %f'%(precision_2,recall_2))
print ('Label 3 Precision: %f, Recall: %f'%(precision_3,recall_3))
print ('Label 4 Precision: %f, Recall: %f'%(precision_4,recall_4))

Test accuracy: 0.992800
Label 0 Precision: 0.996929, Recall: 0.993878
Label 1 Precision: 0.994723, Recall: 0.996476
Label 2 Precision: 0.987366, Recall: 0.984496
Label 3 Precision: 0.988142, Recall: 0.990099
Label 4 Precision: 0.993902, Recall: 0.995927
